# Sample Charts using shared utils

You'll have to look at the docs for the specific charting packages for how to construct the chart. But, these examples show how to layer some pre-defined themes with additional colors to get as close to the FiveThirtyEight theme as possible. 

In [ ]:
import altair as alt
import altair_saver
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns

from plotnine import *

import shared_utils

In [ ]:
TABULAR_DATA = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/taxis.csv"
df = pd.read_csv(TABULAR_DATA)

In [ ]:
df2 = shared_utils.geography_utils.aggregate_by_geography(
    df[(df.payment.notna()) & (df.pickup_borough.notna())], 
    group_cols = ["pickup_borough", "payment"],
    sum_cols = ["passengers", "fare"],
    mean_cols = ["distance"],
    count_cols = [],
    nunique_cols = ["pickup_zone", "dropoff_zone"]
)

df2.head()

## altair example

* altair present themes: https://vega.github.io/vega-themes/
* rendering in notebook is ok, but saving as PNG loses these chart configurations
* use function to apply the `fivethirtyeight` theme again

In [ ]:
alt.themes.enable("fivethirtyeight")

chart = (alt.Chart(df2)
         .mark_bar()
         .encode(
             x=alt.X("pickup_borough:N", title="Origin Borough"),
             y=alt.Y("sum(fare):Q", title="Total Fares"),
             column = "payment:N",
             color = alt.Color("pickup_borough:N", 
                              scale=alt.Scale(
                                  range=shared_utils.altair_utils.FIVETHIRTYEIGHT_DIVERGING_COLORS
                              )
                              )
         )
)

chart = shared_utils.altair_utils.preset_chart_config(chart)

# Add further tweaks to configuration after
chart = (chart
         .properties(width = 100, height = 200, 
                     title="Fares by Payment Type")
        )
chart.save("../example_report/altair_chart.png")
chart

## matplotlib

In [ ]:
plt.style.use('fivethirtyeight')

plt.bar("pickup_borough", height="fare", data=df2)
plt.xlabel("Origin Borough")
plt.ylabel("Total Fares")
plt.title("Fares by Borough")
plt.savefig("../example_report/matplotlib_chart.png")
plt.close()

## seaborn

Set the `fivethirtyeight` color palette

In [ ]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

In [ ]:
sns_plot = (sns.barplot(x=df2.pickup_borough, y=df2.fare, ci=None, 
                       palette=shared_utils.altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS)
            .set_title("Fares by Borough")
           )
chart3 = sns_plot.get_figure()
chart3.tight_layout()
chart3.savefig("../example_report/seaborn_chart.png")

## plotnine

In [ ]:
(ggplot(df2, aes(x='payment', y="distance", fill='payment'))
 + geom_col()
 + theme_538()
 + theme(plot_background=element_rect(fill="white"), 
         panel_background=element_rect(fill="white"), 
        )
 + scale_fill_manual(values=shared_utils.altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS)
 + labs(title = "Distance by Payment Type")
)

## plotly

In [ ]:
fig = px.bar(df2, x="payment", y="distance", 
             color_discrete_sequence=shared_utils.altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS,
             template = "plotly_white",
             title="Distance Traveled by Payment Type"
            )
fig.show()